# Introduction to interactive python profiling

The following notebook will provide a few examples on methods and tools to help profile your python code.
This will help to find bottlenecks and calls which can be optimised.

### Prerequisites

In [ ]:
!pip install line_profiler numba memory_profiler

In [ ]:
# For using lprof
%load_ext line_profiler

In [ ]:
# For using memit and mprun
%load_ext memory_profiler

### Functions
A few functions that we'll be testing later in the code

In [ ]:
def sum_of_numbers(max_value):
    """Sum of integers from 1 to max_value (excluding max_value)"""
    _sum=0
    for i in range(1,max_value):
        _sum += i
    return _sum

In [ ]:
def is_prime(n):
    """Simplest method to determine if a number has any factors"""
    for i in range(2,n):
        if (n%i == 0):
            return False

    return True

def count_primes(n):
    """Return a list of primes found for integer n"""
    primes = []
    for i in range(2,n):
        if is_prime(i) :
            primes.append(i)
    return primes


## Timing

The simplest way to start profiling is just to measure the time a function (or cell) takes to run. 
Then, you can start to delve deeper

In [ ]:
MAX_VAL=1000000

In [ ]:
%time sum_of_numbers(MAX_VAL)

In [ ]:
%time sum(range(MAX_VAL))

### Thoughts
  - Try to use inbuilt functions, when they exist. 
  - Use profiling to understand if / when they do provide a performance boost.
  - Consider refactoring your code/algorithm in terms of inbuilt functions.

Single point estimators may not capture the typical behaviour. use _timeit_ to run a number of trials over your function:
  - Provides a more quantified estimate of your functions runtime
  - Default options will run more/or fewer trials depending on the runtime of the calls

In [ ]:
%%timeit
sum_of_numbers(MAX_VAL)

In [ ]:
%%timeit
sum(range(MAX_VAL))

### Thoughts
Look at the numbers for the timings above and compare _timeit_ to _time_. 
Probably, you see that the calls in _timeit_ are faster than _time_. 
This is because _timeit_ is clever and attempts to limit various system calls which might affect the timing.

For longer running tasks, where fluctations are less relevant, _time_ can be the better measure.

# Primes

Lets consider the example of finding all the prime factors of some number.
We'll go through a few of the simple steps to test and see if we can analyse the slowest / most memory consuming parts of the code, and perhaps see how to improve it.

### Correctness
Lets first convince ourselves that the functions appear to be correct

In [ ]:
for i in range(0,10):
    print (f'n: {i} found primes: {count_primes(i)}')

The above test is sufficient to believe it is not totally wrong. 
Let's formalise this a bit more, with some harder examples, and a unit test.
(note there are several other ways to do this; some using more notebook-based magic functions)

In [ ]:
import unittest

class TestPrimes(unittest.TestCase):
    def test_1(self):
        self.assertEqual(count_primes(1), [])

    def test_3(self):
        self.assertEqual(count_primes(3), [2])

    def test_8(self):
        self.assertEqual(count_primes(8), [2,3,5,7])

    def test_9(self):
        self.assertEqual(count_primes(9), [2,3,5,7])
    
    def test_count_10k(self):
        #https://primes.utm.edu/howmany.html 
        self.assertEqual(len(count_primes(10000)), 1229 )


    def test_float(self):
        # check we error on non-interger number
        self.assertRaises(TypeError, count_primes, 5.5)
    
unittest.main(argv=[''], verbosity=2, exit=False);

Assuming we believe that the above unittests are sufficient (for now) to trust the correctness of the algorithm, lets carry on and try to profile it.

## Profiling

Let's get a feeling for the expected speeds, and perhaps how one might consider it scaling for larger numbers

In [ ]:
%time p = count_primes(10);
%time p = count_primes(100);
%time p = count_primes(1000);
%time p = count_primes(10000);

While _fast enough_ for small n, clearly this is not optimimal for larger n. Let's see where the time is spent.

In [ ]:
pp = %lprun -r  -f count_primes -f is_prime count_primes(10000)
# if you want to printo out the stats:
pp.print_stats()

From the output we see that almost all the time is spent in the _is_prime_ function, and within the function the time is approximately evenly split between the loop, and the test of divisibility. 


### Try:
Perhaps already you can think of some _simple_ changes to the slowest parts of the code to make some reasonable speed improvements?
  - See if you can make some changes to the code to speed up the time. 
  - Make a note of the change, and the speed improvements
  - Optional (plot the time taken as a function of n, for different versions of the algorithm)
  - *Remember* Use the unittests (and improve them if needed) to verify the correctness of any changes, before profiling


In [ ]:
# Use these cells to try and make any changes to the code above and run any profiling.

### Memory profile
It's not just clock cycles that are important; intelligent use of memory can also avoid bottlenecks / breaking of code 

In [ ]:
%memit p = count_primes(10);
%memit p = count_primes(10000);
%memit p = count_primes(30000);

We see that the memory is reasonably constant with (smallist) n, but is also over 100MB (in my case). 
We can profile the functions to see where this memory is allocated.
Unfortunately - for this to happen, we need put the code into it's own module, in order for _mprun_ to work

In [ ]:
%%file mprun_primes.py

def is_prime(n):
    """Simplest method to determine if a number has any factors"""
    for i in range(2,n):
        if (n%i == 0):
            return False

    return True

def count_primes(n):
    """Return a list of primes found for integer n"""
    primes = []
    for i in range(2,n):
        if is_prime(i) :
            primes.append(i)
    return primes



In [ ]:
import mprun_primes
import importlib
importlib.reload(mprun_primes)
from mprun_primes import count_primes,is_prime
#enable module reloading in case of changing the algorithm


In [ ]:
%mprun -f count_primes -f is_prime count_primes(1000)

As this example is not particualrly exceisvie on memory, _optionally_ you might wish to see if better memory usage can be made (an obvious example might be to consider array, or numpy for example)

### Compiling code
While python is an interpreted language, calls can be made to code that has been compiled, potentially allowing for cpu optimisations. 
Code written in c++ (for example) can be made to provide python bindings, but this is not so trivial to do.

Fortunately packages exist (like numbaa) that can help in compiling the code you write in python

Let's write our two functions again, but this time, use the numba decorator _jit_ to get compiled versions of your code.

In [ ]:
from numba import jit
# Use numba to compile the code 

@jit(nopython=True)
def is_prime_jit(n):
    """Simplest method to determine if a number has any factors"""
    for i in range(2,n):
        if (n%i == 0):
            return False

    return True

@jit(nopython=True)
def count_primes_jit(n):
    """Return a list of primes found for integer n"""
    primes = []
    for i in range(2,n):
        if is_prime_jit(i) :
            primes.append(i)
    return primes


Now let's run the simple profile tests again

In [ ]:
%time p = count_primes_jit(10);
%time p = count_primes_jit(100);
%time p = count_primes_jit(1000);
%time p = count_primes_jit(10000);
%time p = count_primes_jit(100000);

Hopefully you observe a significant speedup.
 - The first time you run the method, there is an overhead in compiling the code
 - Note, that profiling on a per-line basis e.g. with lprun no longer returns useful values

#### Try:
 - Optional - consider thinking about the problem and making any further changes to how the code might run. 
 - Optional - how do you expect / understand the time to scale with n

### Changing Algorithm
As mentioned in the lecture, at some point, the largest gains are likely to be made from changing the algorithm itself.
Below are two versions available from https://stackoverflow.com/questions/2068372/fastest-way-to-list-all-primes-below- which have had significant effort made to be optimal, without using tools outside of standard python. 

Lets see how they perform ... 

In [ ]:
#source: https://stackoverflow.com/questions/2068372/fastest-way-to-list-all-primes-below-n
from itertools import compress

def rwh_primes1v1(n):
    """ Returns  a list of primes < n for n > 2 """
    sieve = bytearray([True]) * (n//2)
    for i in range(3,int(n**0.5)+1,2):
        if sieve[i//2]:
            sieve[i*i//2::i] = bytearray((n-i*i-1)//(2*i)+1)
    return [2,*compress(range(3,n,2), sieve[1:])]

def rwh_primes1v2(n):
    """ Returns a list of primes < n for n > 2 """
    sieve = bytearray([True]) * (n//2+1)
    for i in range(1,int(n**0.5)//2+1):
        if sieve[i]:
            sieve[2*i*(i+1)::2*i+1] = bytearray((n//2-2*i*(i+1))//(2*i+1)+1)
    return [2,*compress(range(3,n,2), sieve[1:])]


In [ ]:
MAX_PRIMES=100000

In [ ]:
%time primes_basic = count_primes(MAX_PRIMES)

In [ ]:
%time primes_1v1 = rwh_primes1v1(MAX_PRIMES)

In [ ]:
%time primes_1v2 = rwh_primes1v2(MAX_PRIMES)

In [ ]:
%time primes_jit = count_primes_jit(MAX_PRIMES)

Did we test for Correctness ? 

We can use our existing code for comparison. 

Would these new functions pass our unittests above? Think, and _optionally_ try to create/run the same unittest for these new functions. 
  - what would be needed, and what's the performance overhead to ensure identical outputs?

In [ ]:
print(set(primes_1v1) == set(primes_1v2))
print(set(primes_basic) == set(primes_jit))
print(set(primes_basic) == set(primes_1v1))

While we have only touched the surface, on a very broad and detail topic; hopefully you now have a basic understanding of some of the tools that can be quickly used to profile your own functions in python. 

Experiment with profiling whenever you have a slow function, and consider tests to help keep your code bug-free.

Thanks for making it this far!